# Prefix-tuning sentiment classification 예제 - sentences_allagree 데이터셋
input으로 금융 뉴스에 대한 텍스트. 경제 뉴스가 positive, negative, netrual인지 분류
## Reference : https://huggingface.co/docs/peft/task_guides/seq2seq-prefix-tuning

# 필요한 라이브러리 설치

In [1]:
!pip install -q peft transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# 설정값 지정

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, default_data_collator, get_linear_schedule_with_warmup
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, PrefixTuningConfig, TaskType # peft 라이브러리의 prefixtuningconfig 모듈 불러오기
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

device = "cuda"
model_name_or_path = "t5-large" # 자연어처리 모델
tokenizer_name_or_path = "t5-large"

text_column = "sentence"
label_column = "text_label"
max_length = 128
lr = 1e-2 # 0.01
num_epochs = 5
batch_size = 8

# sentences_allagree 데이터셋 불러오기
## (financial_phrasebank 데이터셋의 부분집합 데이터셋으로 **금융 뉴스의 감정(sentiment)를 분류**하는 데이터셋입니다.)

In [3]:
from datasets import load_dataset

dataset = load_dataset("financial_phrasebank", "sentences_allagree")
dataset = dataset["train"].train_test_split(test_size=0.1) # 90%는 training, 10%는 test로 나누기
dataset["validation"] = dataset["test"]
del dataset["test"]

classes = dataset["train"].features["label"].names
dataset = dataset.map(
    lambda x: {"text_label": [classes[label] for label in x["label"]]},
    batched=True,
    num_proc=1,
)

dataset["train"][0] #트레이닝 데이터 뉴스 하나 보기

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.88k [00:00<?, ?B/s]

financial_phrasebank.py:   0%|          | 0.00/6.04k [00:00<?, ?B/s]

The repository for financial_phrasebank contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/financial_phrasebank.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


FinancialPhraseBank-v1.0.zip:   0%|          | 0.00/682k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2264 [00:00<?, ? examples/s]

Map:   0%|          | 0/2037 [00:00<?, ? examples/s]

Map:   0%|          | 0/227 [00:00<?, ? examples/s]

{'sentence': "Poyry 's contract includes engineering management , civil and detail engineering services , and time scheduling and procurement services .",
 'label': 1,
 'text_label': 'neutral'}

# Tokenizer 불러오기 & 데이터셋 전처리

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

def preprocess_function(examples):
    inputs = examples[text_column]
    targets = examples[label_column]
    model_inputs = tokenizer(inputs, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")
    labels = tokenizer(targets, max_length=2, padding="max_length", truncation=True, return_tensors="pt")
    labels = labels["input_ids"]
    labels[labels == tokenizer.pad_token_id] = -100
    model_inputs["labels"] = labels

    return model_inputs

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [5]:
processed_datasets = dataset.map( # 정리해놓은 preprocess 함수로 processing 작업해서 데이터 전처리 해준 다음에
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset:   0%|          | 0/2037 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/227 [00:00<?, ? examples/s]

In [6]:
train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["validation"]

train_dataloader = DataLoader( # 트레이닝 데이터랑 벨리데이션 데이터를 배치사이즈로 묶어주기
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)
eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)

# PEFT 모델 설정

In [7]:
peft_config = PrefixTuningConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, num_virtual_tokens=20) # num_virtual_token은 프리픽스 길이를 얼만큼 할건지

model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

# Prefix-Tuning 기법으로 인해 전체 모델의 0.13%의 파라미터만 Fine-Tuning에 사용

In [8]:
model = get_peft_model(model, peft_config) # 앞에는 프리트레이닝된 모델을 넣고 두번째는 지정해놓은 peftconfig 지정하면 프리픽스 튜닝형태로 모델 적용
model.print_trainable_parameters() # 학습해야할 파라미터 양이 얼마나 줄었는지 확인

trainable params: 983,040 || all params: 738,651,136 || trainable%: 0.1331


In [9]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

# Training 시작

In [10]:
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")


100%|██████████| 29/29 [00:07<00:00,  3.75it/s]


epoch=0: train_ppl=tensor(5.0342, device='cuda:0') train_epoch_loss=tensor(1.6163, device='cuda:0') eval_ppl=tensor(1.0928, device='cuda:0') eval_epoch_loss=tensor(0.0888, device='cuda:0')


100%|██████████| 29/29 [00:07<00:00,  3.93it/s]


epoch=1: train_ppl=tensor(1.1222, device='cuda:0') train_epoch_loss=tensor(0.1153, device='cuda:0') eval_ppl=tensor(1.0446, device='cuda:0') eval_epoch_loss=tensor(0.0436, device='cuda:0')


100%|██████████| 29/29 [00:07<00:00,  4.12it/s]


epoch=2: train_ppl=tensor(1.0867, device='cuda:0') train_epoch_loss=tensor(0.0831, device='cuda:0') eval_ppl=tensor(1.0412, device='cuda:0') eval_epoch_loss=tensor(0.0403, device='cuda:0')


100%|██████████| 29/29 [00:07<00:00,  3.97it/s]


epoch=3: train_ppl=tensor(1.0768, device='cuda:0') train_epoch_loss=tensor(0.0740, device='cuda:0') eval_ppl=tensor(1.0594, device='cuda:0') eval_epoch_loss=tensor(0.0577, device='cuda:0')


100%|██████████| 29/29 [00:07<00:00,  4.11it/s]

epoch=4: train_ppl=tensor(1.0678, device='cuda:0') train_epoch_loss=tensor(0.0656, device='cuda:0') eval_ppl=tensor(1.0371, device='cuda:0') eval_epoch_loss=tensor(0.0364, device='cuda:0')


# 학습이 끝난 모델 성능 평가

In [11]:
correct = 0
total = 0
for pred, true in zip(eval_preds, dataset["validation"]["text_label"]):
    if pred.strip() == true.strip():
        correct += 1
    total += 1
accuracy = correct / total * 100
print(f"{accuracy=} % on the evaluation dataset")
print(f"{eval_preds[:10]=}")
print(f"{dataset['validation']['text_label'][:10]=}")

accuracy=98.23788546255507 % on the evaluation dataset
eval_preds[:10]=['positive', 'neutral', 'negative', 'negative', 'positive', 'positive', 'neutral', 'negative', 'neutral', 'negative']
dataset['validation']['text_label'][:10]=['positive', 'neutral', 'negative', 'negative', 'positive', 'positive', 'neutral', 'negative', 'neutral', 'negative']


# 학습이 끝난 모델을 sample text에 대한 Inference

In [12]:
inputs = tokenizer(
    "The Lithuanian beer market made up 14.41 million liters in January , a rise of 0.8 percent from the year-earlier figure , the Lithuanian Brewers ' Association reporting citing the results from its members .",
    return_tensors="pt",
)

In [13]:
with torch.no_grad():
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))

['positive . positive']
